In [ ]:
!pip install -q evaluate transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 26.3 MB/s eta 0:00:00


In [ ]:
# Load a dataset from Hugging Face
from datasets import load_dataset

hf_dataset_identifier = "jacquelinegrimm/arabidopsis-roots-multi"
ds = load_dataset(hf_dataset_identifier)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
# Shuffle and split the dataset into training and test sets

ds = ds.shuffle(seed=1) # The 'seed' parameter is set to 1 for reproducibility
ds = ds["train"].train_test_split(test_size=0.2)

train_ds = ds["train"]
test_ds = ds["test"]

In [ ]:
import json
from huggingface_hub import hf_hub_download

In [ ]:
# To configure the model, extract the number of unique labels

repo_id = f"datasets/{hf_dataset_identifier}"
filename = "id2label.json" # Define the filename containing label information

# Download the 'id2label.json' file
id2label = json.load(open(hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"), "r"))

# Convert the keys in 'id2label' from strings to integers and store the result in 'id2label'
id2label = {int(k): v for k, v in id2label.items()}

# Create a 'label2id' dictionary by reversing the key-value pairs in 'id2label'
label2id = {v: k for k, v in id2label.items()}

# Calculate the number of unique labels in the dataset
num_labels = len(id2label)

id2label.json:   0%|          | 0.00/89.0 [00:00<?, ?B/s]

In [ ]:
from torchvision.transforms import ColorJitter
from transformers import SegformerImageProcessor

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# Transform data to match the model's expected input shape and perform image augmentation
# Apply color jitter, which introduces random variations in brightness, contrast, saturation, and hue

processor = SegformerImageProcessor()
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

# Function to apply transformations to a batch of training examples
def train_transforms(example_batch):

    # Apply color jitter to each image in the batch
    images = [jitter(x) for x in example_batch['pixel_values']]
    # Extract labels from the batch
    labels = [x for x in example_batch['label']]
    # Use the processor to transform images and labels to match the expected input shape
    inputs = processor(images, labels)
    # Return the transformed inputs
    return inputs

# Function to apply transformations to a batch of validation examples
def val_transforms(example_batch):

    # Extract images from the batch without applying color jitter
    images = [x for x in example_batch['pixel_values']]
    # Extract labels from the batch
    labels = [x for x in example_batch['label']]
    # Use the processor to transform images and labels to match the expected input shape
    inputs = processor(images, labels)
    # Return the transformed inputs
    return inputs

# Set the transformation functions for the training and testing datasets
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [ ]:
# Load the semantic segmentation model
from transformers import SegformerForSemanticSegmentation

pretrained_model_name = "nvidia/mit-b0"

model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    id2label=id2label,
    label2id=label2id
)

config.json:   0%|          | 0.00/70.0k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/14.4M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.wei

In [ ]:
# Specify training arguments and name the fine-tuned model
from transformers import TrainingArguments

# Define training hyperparameters
epochs = 50
lr = 0.00006
batch_size = 2

hub_model_id = "segformer-b0-finetuned-arabidopsis-roots-multi" # Name the fine-tuned model

training_args = TrainingArguments(
    "segformer-b0-finetuned-arabidopsis-roots-multi-outputs",  # Name the output directory
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id=hub_model_id,
    hub_strategy="end",
)

In [ ]:
# Define mean intersection over union (IoU) as the evaluation metric
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")

# Function to calculate evaluation metrics for model predictions
def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred

        # Convert 'logits' into a tensor and resize it to match the shape of "labels"
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        # Convert the predicted labels to a NumPy array
        pred_labels = logits_tensor.detach().cpu().numpy()

        # Calculate metrics using the 'mean_iou' evaluation metric
        metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),  # Number of unique labels in the dataset
            ignore_index=0,
            reduce_labels=processor.do_reduce_labels,
        )

        # Extract per-category accuracy and IoU scores
        per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
        per_category_iou = metrics.pop("per_category_iou").tolist()

        # Update the metrics dictionary with accuracy and IoU scores for each category
        metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
        metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

        return metrics

In [ ]:
# Log in to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Train the model
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Background,Accuracy Main root,Accuracy Lateral root,Accuracy Shoot,Accuracy Botrytis,Iou Background,Iou Main root,Iou Lateral root,Iou Shoot,Iou Botrytis
20,1.183200,1.234694,0.338316,0.495216,0.479919,nan,0.161407,0.776407,0.174718,0.868333,0.000000,0.137680,0.535494,0.170046,0.848361
40,0.901000,0.786448,0.305073,0.410467,0.343752,nan,0.006167,0.393307,0.316575,0.925820,0.000000,0.006121,0.323835,0.285327,0.910082
60,0.716100,0.638621,0.305927,0.413312,0.342974,nan,0.027116,0.338938,0.334923,0.952273,0.000000,0.026978,0.291927,0.284657,0.926071
80,0.635800,0.602070,0.315546,0.422469,0.350154,nan,0.038997,0.333806,0.371307,0.945767,0.000000,0.038706,0.287846,0.324060,0.927115
100,0.511100,0.477042,0.287065,0.377892,0.317455,nan,0.056659,0.288007,0.279776,0.887124,0.000000,0.056162,0.249274,0.257109,0.872779
120,0.484200,0.406932,0.242234,0.323241,0.279206,nan,0.114247,0.255855,0.373380,0.549482,0.000000,0.111165,0.228986,0.323994,0.547023
140,0.363100,0.352672,0.269219,0.358140,0.353712,nan,0.338208,0.341121,0.335286,0.417944,0.000000,0.316776,0.302118,0.309255,0.417944
160,0.331600,0.298600,0.423936,0.573127,0.549562,nan,0.471228,0.500273,0.530579,0.790429,0.000000,0.437190,0.433299,0.471903,0.777287
180,0.307400,0.275514,0.375520,0.506121,0.493217,nan,0.443953,0.487636,0.520214,0.572682,0.000000,0.412875,0.428061,0.467947,0.568714
200,0.321600,0.231672,0.426230,0.576200,0.562545,nan,0.567565,0.465555,0.585311,0.686371,0.000000,0.515294,0.420558,0.514230,0.681065


/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encoun

TrainOutput(global_step=1000, training_loss=0.21457392946258189, metrics={'train_runtime': 2964.0077, 'train_samples_per_second': 0.675, 'train_steps_per_second': 0.337, 'total_flos': 3.5063187111936e+16, 'train_loss': 0.21457392946258189, 'epoch': 50.0})

In [ ]:
# Push the fine-tuned model to Hugging Face
kwargs = {
    "tags": ["vision", "image-segmentation"],
    "finetuned_from": pretrained_model_name,
    "dataset": hf_dataset_identifier,
}

processor.push_to_hub(hub_model_id)
trainer.push_to_hub(**kwargs)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1709067671.b9c7c2286e97.295.0:   0%|          | 0.00/214k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jacquelinegrimm/segformer-b0-finetuned-arabidopsis-roots-multi/commit/9596a2663c32ae3efb81a5190a3c04610053c649', commit_message='End of training', commit_description='', oid='9596a2663c32ae3efb81a5190a3c04610053c649', pr_url=None, pr_revision=None, pr_num=None)